In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd  PATH

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
import torchvision.models as models
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm

import torch.nn.functional as F
import csv
from torchvision.datasets import DatasetFolder, ImageFolder
from PIL import Image
import glob as gb

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((256,256)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.5,contrast=0,saturation =0,hue =0),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
    'val': transforms.Compose([
        transforms.Resize((256,256) ),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
}

data_dir = './training'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
image_datasets2 = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms['val']) for x in ['train']}
image_datasets['train'] = ConcatDataset([image_datasets['train'],image_datasets2['train']])

batch_size = 32
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

inputs, classes = next(iter(dataloaders['train']))

print(f"Using device {device}\n")

In [ ]:
losses = {'train':[], 'val':[]}
accuracies = {'train':[], 'val':[]}
lr = []

In [ ]:
# Resnet50、resnext50_32x4d、googlenet、efficientnet_b4

def train_classifier(seed, epochs, model, since):
    print('Creating a model {}...'.format(seed))

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    if seed==0 or seed==1 or seed==2:
      # optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay = 1e-5)
      optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)
    else:
      # optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
      optimizer = optim.SGD(model.classifier.parameters(), lr=0.01, momentum=0.9)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(epochs):
      for phase in ['train', 'val']:
        if phase == 'train':
          model.train()
        else:
          model.eval()
        
        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in tqdm(dataloaders[phase]):
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()

          with torch.set_grad_enabled(phase=='train'):
            outp = model(inputs)
            _, pred = torch.max(outp, 1)
            loss = criterion(outp, labels)
          
            if phase == 'train':
              loss.backward()
              optimizer.step()
          running_loss += loss.item()*inputs.size(0)
          running_corrects += torch.sum(pred == labels.data)

        if phase == 'train':
            acc = 100. * running_corrects.double() / dataset_sizes[phase]
            scheduler.step(acc)

        epoch_loss = (running_loss / dataset_sizes[phase])
        epoch_acc = (running_corrects.double()/dataset_sizes[phase])
        losses[phase].append(epoch_loss)
        accuracies[phase].append(epoch_acc)
        if phase == 'train':
          print('Epoch: {}/{}'.format(epoch+1, epochs))
        print('{} - loss:{:.4f}, accuracy: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        lr.append(scheduler._last_lr)
          
        if phase == 'val':
          print('Time: {}m {:.2f}s'.format((time.time()- since)//60, (time.time()- since)%60))
          print('=='*31)
        if phase == 'val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('CLASSIFIER TRAINING TIME {}m {:.2f}s'.format(time_elapsed//60, time_elapsed%60))
    print('=='*31)
    print(f'Best Acc of model{seed}: {best_acc}')
    return best_model


In [ ]:
def train_net(seed, epochs, model, since):
  model.to(device)
  for param in model.parameters():
        param.requires_grad=True
  
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)  
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=True)
  best_model = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(epochs):
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()
      
      running_loss = 0.0
      running_corrects = 0.0

      for inputs, labels in tqdm(dataloaders[phase]):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase=='train'):
          outp = model(inputs)
          _, pred = torch.max(outp, 1)
          loss = criterion(outp, labels)
        
          if phase == 'train':
            loss.backward()
            optimizer.step()

        running_loss += loss.item()*inputs.size(0)
        running_corrects += torch.sum(pred == labels.data)

      if phase == 'train':
        acc = 100. * running_corrects.double() / dataset_sizes[phase]
        scheduler.step(acc)

      epoch_loss = (running_loss / dataset_sizes[phase])
      epoch_acc = (running_corrects.double()/dataset_sizes[phase])
      losses[phase].append(epoch_loss)
      accuracies[phase].append(epoch_acc)
      if phase == 'train':
        print('Epoch: {}/{}'.format(epoch+1, epochs))
      print('{} - loss:{:.4f}, accuracy: {:.4f}'.format(phase, epoch_loss, epoch_acc))
      lr.append(scheduler._last_lr)
    
      if phase == 'val':
        print('Time: {}m {}s'.format((time.time()- since)//60, (time.time()- since)%60))
        print('=='*31)    
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model = copy.deepcopy(model.state_dict())
  time_elapsed = time.time() - since
  print('ALL NET TRAINING TIME {}m {:.2f}s'.format(time_elapsed//60, time_elapsed%60))
  print('=='*31)
  print(f'Best Acc of model{seed}: {best_acc}')

  model.load_state_dict(best_model)
  torch.save(model.state_dict(), f'./model{seed}.pth')
  return model

In [ ]:
def train(seed, epochs, model):
  since = time.time()

  best_model = train_classifier(seed, epochs, model, since)
  model.load_state_dict(best_model)
  model = train_net(seed, epochs, model, since)
  
  return model

In [ ]:
resnet50 = torchvision.models.resnet50(pretrained=True)
for param in resnet50.parameters():
  param.grad_requires = False

resnet50.fc = nn.Linear(in_features=resnet50.fc.in_features, out_features=219, bias=True)


resnext50_32x4d = torchvision.models.resnext50_32x4d(pretrained=True)
for param in resnext50_32x4d.parameters():
  param.grad_requires = False

resnext50_32x4d.fc = nn.Linear(in_features=resnext50_32x4d.fc.in_features, out_features=219, bias=True)


efficientnet_b4 = torchvision.models.efficientnet_b4(pretrained=True)
for param in efficientnet_b4.parameters():
  param.grad_requires = False

efficientnet_b4.classifier[-1] = nn.Linear(in_features=efficientnet_b4.classifier[-1].in_features, out_features=219, bias=True)

convnext_base = torchvision.models.convnext_base(pretrained=True)
for param in convnext_base.parameters():
  param.grad_requires = False

convnext_base.classifier[-1] = nn.Linear(in_features=convnext_base.classifier[-1].in_features, out_features=219, bias=True)

epochs = 10

models = [resnet50, resnext50_32x4d, efficientnet_b4, convnext_base]
num_models = len(models)

for seed in range(num_models):
   train(seed=seed, epochs=epochs, model=models[seed])
   losses = {'train':[], 'val':[]}
   accuracies = {'train':[], 'val':[]}
   lr = []

In [ ]:
class Ensemble1(nn.Module):
    def __init__(self, models_list, device):
        super(Ensemble1,self).__init__()
        self.models = nn.ModuleList(models_list)
        
    def forward(self, x):
        output = torch.zeros([x.size(0), 219]).to(device)
        for model in self.models:
            output += model(x)
        return output

In [ ]:
resnet50 = torchvision.models.resnet50()
resnet50.fc = nn.Linear(in_features=resnet50.fc.in_features, out_features=219, bias=True)
resnet50.load_state_dict(torch.load(f'./ensemble/model0.pth'))

resnext50_32x4d = torchvision.models.resnext50_32x4d()
resnext50_32x4d.fc = nn.Linear(in_features=resnext50_32x4d.fc.in_features, out_features=219, bias=True)
resnext50_32x4d.load_state_dict(torch.load(f'./ensemble/model1.pth'))

densenet201 = torchvision.models.densenet201()
densenet201.classifier = nn.Linear(in_features=densenet201.classifier.in_features, out_features=219, bias=True)
densenet201.load_state_dict(torch.load(f'./ensemble/model3.pth'))

convnext_base = torchvision.models.convnext_base()
convnext_base.classifier[-1] = nn.Linear(in_features=convnext_base.classifier[-1].in_features, out_features=219, bias=True)
convnext_base.load_state_dict(torch.load(f'./ensemble/model4.pth'))


models_list = [resnet50, resnext50_32x4d, densenet201, convnext_base]


ensemble_model = Ensemble1(models_list, device)
ensemble_model.to(device)

In [ ]:
test_tfm = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

dataset=datasets.ImageFolder('./data/',transforms.Compose([
      transforms.Resize((256,256) ),
      transforms.ToTensor(),
      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
  ]),)

filename=[]

for i in range(len(dataset.imgs)):
  y = dataset.imgs[i][0]
  y = str(y).replace('./data/train_data/','')
  filename.append(y)
    
test_loader = DataLoader(dataset, batch_size=8, shuffle=False)
model = ensemble_model
model.to(device)
model.eval()


predictions = []
for batch in tqdm(test_loader):
  imgs, labels = batch
  with torch.no_grad():
      inputs = imgs.to(device)
      logits = model(inputs)
  predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())